In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


# pd.set_option('max_columns', 50)
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

from prophet import Prophet
import os
import warnings
warnings.filterwarnings('ignore')

# class cfg():
    
#     notebook_name = 'Prophet Baseline'
#     data_dir = '/kaggle/input/gdz-elektrik-datathon-2023'
    
#     dates = ['Tarih']
#     target = 'Dağıtılan Enerji (MWh)'

In [2]:
train = pd.read_csv('train.csv' , parse_dates=['Tarih'])
med = pd.read_csv('med.csv' , parse_dates=['Tarih'])
sub = pd.read_csv('sample_submission.csv' , parse_dates=['Tarih'])
future_data = pd.read_csv("future.csv").drop('Unnamed: 0' , axis = 1)
test = sub.copy()

full_df = pd.concat([
    train,
    test
], axis=0)

all_data = [train , test]

In [3]:
# color_pal = ["#F8766D", "#D39200", "#93AA00",
#              "#00BA38", "#00C19F", "#00B9E3",
#              "#619CFF", "#DB72FB"]
# train.set_index(cfg.dates[0]).plot(style='.', figsize=(15,5), color=color_pal[0])
# plt.show()

In [4]:
from feat_engs import create_datetimes, create_businessDay_cumulativeholidays ,create_electricOutage_timeofDay , create_holiday_weekend , create_hourly_sin_cos , create_outage_rolling_percentages , create_seasons

for df in all_data:
    create_datetimes(df)
    create_holiday_weekend(df)
    create_businessDay_cumulativeholidays(df)
    create_electricOutage_timeofDay(df , med)
    create_hourly_sin_cos(df)
    # create_outage_rolling_percentages(df)
    create_seasons(df)
    # train.set_index('Tarih' , inplace=True)
    df.drop('date' , axis = 1 ,inplace=True)

In [5]:
# def create_features(df, label=None):
#     df['date'] = df.index
#     df['hour'] = df['date'].dt.hour
#     df['dayofweek'] = df['date'].dt.dayofweek
#     df['quarter'] = df['date'].dt.quarter
#     df['month'] = df['date'].dt.month
#     df['year'] = df['date'].dt.year
#     df['dayofyear'] = df['date'].dt.dayofyear
#     df['dayofmonth'] = df['date'].dt.day
#     df['weekofyear'] = df['date'].dt.weekofyear
    
#     X = df[['hour','dayofweek','quarter','month','year',
#            'dayofyear','dayofmonth','weekofyear']]
#     if label:
#         y = df[label]
#         return X, y
#     return X

# X, y = create_features(train.set_index('Tarih'), label=['Dağıtılan Enerji (MWh)'])

# features_and_target = pd.concat([X, y], axis=1)

In [6]:
# sns.pairplot(features_and_target.dropna(),
#              hue='hour',
#              x_vars=['hour','dayofweek',
#                      'year','weekofyear'],
#              y_vars='Dağıtılan Enerji (MWh)',
#              height=5,
#              plot_kws={'alpha':0.15, 'linewidth':0}
#             )
# plt.suptitle('Energy by Hour, Day of Week, Year and Week of Year')
# plt.show()

### Remove Power Outage

In [7]:
train.shape

(40152, 24)

In [8]:
med['isover'] = True
train = pd.merge(train, 
                 med,
                 how='left',
                 on='Tarih')
# train = train[train['isover'].isnull()].drop('isover', axis=1)

In [9]:
train.drop(['is_weekend' , 'is_weekday' , 'isover'] , axis  =1 , inplace= True)

### External Data (holidays)

In [10]:
hol = pd.read_csv('Calendar.csv', parse_dates=['CALENDAR_DATE'])
hol.head()

,CALENDAR_SK,CALENDAR_DATE,DAY_OF_MONTH,DAY_OF_WEEK_SK,SEASON_SK,QUARTER_OF_YEAR,MONTH_SK,WEEKEND_FLAG,WEEK_OF_YEAR,RAMADAN_FLAG,RELIGIOUS_DAY_FLAG_SK,NATIONAL_DAY_FLAG_SK,PUBLIC_HOLIDAY_FLAG,SPECIAL_DAY_SK,SPECIAL_DAY_SK2
0,20241231,2024-12-31,31,2,4,4,202412,N,53,N,100,200,N,NaN,NaN
1,20241230,2024-12-30,30,1,4,4,202412,N,53,N,100,200,N,NaN,NaN
2,20241229,2024-12-29,29,7,4,4,202412,Y,53,N,100,200,Y,NaN,NaN
3,20241228,2024-12-28,28,6,4,4,202412,Y,52,N,100,200,Y,NaN,NaN
4,20241227,2024-12-27,27,5,4,4,202412,N,52,N,100,200,N,NaN,NaN


In [11]:
hol = hol[['CALENDAR_DATE','RAMADAN_FLAG','PUBLIC_HOLIDAY_FLAG']].rename(columns={'CALENDAR_DATE':'ds'})
hol.head()

,ds,RAMADAN_FLAG,PUBLIC_HOLIDAY_FLAG
0,2024-12-31,N,N
1,2024-12-30,N,N
2,2024-12-29,N,Y
3,2024-12-28,N,Y
4,2024-12-27,N,N


In [12]:
hol['holiday'] = np.where((hol['RAMADAN_FLAG'] == 'Y') | (hol['PUBLIC_HOLIDAY_FLAG'] == 'Y'), 'TR-Holidays', 0)
hol.head()

,ds,RAMADAN_FLAG,PUBLIC_HOLIDAY_FLAG,holiday
0,2024-12-31,N,N,0
1,2024-12-30,N,N,0
2,2024-12-29,N,Y,TR-Holidays
3,2024-12-28,N,Y,TR-Holidays
4,2024-12-27,N,N,0


In [13]:
hol = pd.read_csv('Calendar.csv', parse_dates=['CALENDAR_DATE'])
hol = hol[['CALENDAR_DATE','RAMADAN_FLAG','PUBLIC_HOLIDAY_FLAG']].rename(columns={'CALENDAR_DATE':'ds'})
hol['holiday'] = np.where((hol['RAMADAN_FLAG'] == 'Y') | (hol['PUBLIC_HOLIDAY_FLAG'] == 'Y'), 'TR-Holidays', 0)
hol = hol[['ds','holiday']]
hol = hol[hol['holiday'] == 'TR-Holidays']

### Target Transformation

In [14]:
train['Dağıtılan Enerji (MWh)'] = np.log1p(train['Dağıtılan Enerji (MWh)'])

### External

In [15]:
# def is_spring(ds):
#     date = pd.to_datetime(ds)
#     return (date.month >= 3) & (date.month <= 5)

# def is_summer(ds):
#     date = pd.to_datetime(ds)
#     return (date.month >= 6) & (date.month <= 8)

# def is_autumn(ds):
#     date = pd.to_datetime(ds)
#     return (date.month >= 9) & (date.month <= 11)

# def is_winter(ds):
#     date = pd.to_datetime(ds)
#     return (date.month >= 12) | (date.month <= 2)

# def is_weekend(ds):
#     date = pd.to_datetime(ds)
#     return date.day_name in ('Saturday', 'Sunday')

# # adding to train set
# train['is_spring'] = train['Tarih'].apply(is_spring)
# train['is_summer'] = train['Tarih'].apply(is_summer)
# train['is_autumn'] = train['Tarih'].apply(is_autumn)
# train['is_winter'] = train['Tarih'].apply(is_winter)
# train['is_weekend'] = train['Tarih'].apply(is_weekend)
# train['is_weekday'] = ~train['Tarih'].apply(is_weekend)

# # adding to test set
# test['is_spring'] = test['Tarih'].apply(is_spring)
# test['is_summer'] = test['Tarih'].apply(is_summer)
# test['is_autumn'] = test['Tarih'].apply(is_autumn)
# test['is_winter'] = test['Tarih'].apply(is_winter)
# test['is_weekend'] = test['Tarih'].apply(is_weekend)
# test['is_weekday'] = ~test['Tarih'].apply(is_weekend)

In [16]:
# train[train['is_winter'] == True]

In [17]:
train.head()

,Tarih,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,...,business_day,cumulative_holidays,electrical_outage,timeofday,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter
0,2018-01-01 00:00:00,7.374594,0,0,1,1,2018,1,1,1,...,1,1,0,3,0.000000,1.000000,0,0,0,0
1,2018-01-01 01:00:00,7.323127,1,0,1,1,2018,1,1,1,...,1,2,0,3,0.258819,0.965926,0,0,0,0
2,2018-01-01 02:00:00,7.246805,2,0,1,1,2018,1,1,1,...,1,3,0,3,0.500000,0.866025,0,0,0,0
3,2018-01-01 03:00:00,7.154246,3,0,1,1,2018,1,1,1,...,1,4,0,3,0.707107,0.707107,0,0,0,0
4,2018-01-01 04:00:00,7.107997,4,0,1,1,2018,1,1,1,...,1,5,0,3,0.866025,0.500000,0,0,0,0


In [18]:
# train[train.isover == True]

In [19]:
# def energy_increase(x):
#     if x > 2000 :
#         return (x+50)
#     elif x > 1800 :
#         return (x+100)
#     elif x > 1500 :
#         return (x+200)
#     else :
#         return (x+350)

# train[train.isover == True]['Dağıtılan Enerji (MWh)']

In [20]:
train

,Tarih,Dağıtılan Enerji (MWh),hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,...,business_day,cumulative_holidays,electrical_outage,timeofday,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter
0,2018-01-01 00:00:00,7.374594,0,0,1,1,2018,1,1,1,...,1,1,0,3,0.000000,1.000000,0,0,0,0
1,2018-01-01 01:00:00,7.323127,1,0,1,1,2018,1,1,1,...,1,2,0,3,0.258819,0.965926,0,0,0,0
2,2018-01-01 02:00:00,7.246805,2,0,1,1,2018,1,1,1,...,1,3,0,3,0.500000,0.866025,0,0,0,0
3,2018-01-01 03:00:00,7.154246,3,0,1,1,2018,1,1,1,...,1,4,0,3,0.707107,0.707107,0,0,0,0
4,2018-01-01 04:00:00,7.107997,4,0,1,1,2018,1,1,1,...,1,5,0,3,0.866025,0.500000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40147,2022-07-31 19:00:00,7.821438,19,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.965926,0.258819,0,1,0,0
40148,2022-07-31 20:00:00,7.816400,20,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.866025,0.500000,0,1,0,0
40149,2022-07-31 21:00:00,7.818437,21,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.707107,0.707107,0,1,0,0
40150,2022-07-31 22:00:00,7.782213,22,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.500000,0.866025,0,1,0,0


In [21]:
train.rename(columns={'Tarih':'ds',
                                'Dağıtılan Enerji (MWh)':'y'})

,ds,y,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,...,business_day,cumulative_holidays,electrical_outage,timeofday,hour_sin,hour_cos,is_spring,is_summer,is_autumn,is_winter
0,2018-01-01 00:00:00,7.374594,0,0,1,1,2018,1,1,1,...,1,1,0,3,0.000000,1.000000,0,0,0,0
1,2018-01-01 01:00:00,7.323127,1,0,1,1,2018,1,1,1,...,1,2,0,3,0.258819,0.965926,0,0,0,0
2,2018-01-01 02:00:00,7.246805,2,0,1,1,2018,1,1,1,...,1,3,0,3,0.500000,0.866025,0,0,0,0
3,2018-01-01 03:00:00,7.154246,3,0,1,1,2018,1,1,1,...,1,4,0,3,0.707107,0.707107,0,0,0,0
4,2018-01-01 04:00:00,7.107997,4,0,1,1,2018,1,1,1,...,1,5,0,3,0.866025,0.500000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40147,2022-07-31 19:00:00,7.821438,19,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.965926,0.258819,0,1,0,0
40148,2022-07-31 20:00:00,7.816400,20,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.866025,0.500000,0,1,0,0
40149,2022-07-31 21:00:00,7.818437,21,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.707107,0.707107,0,1,0,0
40150,2022-07-31 22:00:00,7.782213,22,6,3,7,2022,212,31,30,...,0,1632,0,3,-0.500000,0.866025,0,1,0,0


### Modelling

In [22]:
model = Prophet(holidays=hol)

model.add_seasonality(name='yearly', period=365.27, fourier_order=10)
model.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_spring')
model.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_summer')
model.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_autumn')
model.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_winter')
model.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_spring')
model.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_summer')
model.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_autumn')
model.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_winter')

model.fit(train.rename(columns={'Tarih':'ds',
                                'Dağıtılan Enerji (MWh)':'y'}))

02:21:20 - cmdstanpy - INFO - Chain [1] start processing
02:22:04 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
test_preds = model.predict(df=test.rename(columns={'Tarih':'ds',
                                                   'Dağıtılan Enerji (MWh)':'y'}))
test_preds.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,TR-Holidays,TR-Holidays_lower,TR-Holidays_upper,additive_terms,...,weekly_winter,weekly_winter_lower,weekly_winter_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-08-01 00:00:00,7.525168,7.500742,7.704287,7.525168,7.525168,0.0,0.0,0.0,0.077416,...,0.0,0.0,0.0,0.209828,0.209828,0.209828,0.0,0.0,0.0,7.602585
1,2022-08-01 01:00:00,7.525165,7.416670,7.631448,7.525165,7.525165,0.0,0.0,0.0,0.000809,...,0.0,0.0,0.0,0.209918,0.209918,0.209918,0.0,0.0,0.0,7.525975
2,2022-08-01 02:00:00,7.525162,7.380568,7.576535,7.525162,7.525162,0.0,0.0,0.0,-0.048395,...,0.0,0.0,0.0,0.210008,0.210008,0.210008,0.0,0.0,0.0,7.476767
3,2022-08-01 03:00:00,7.525159,7.359199,7.550078,7.525159,7.525159,0.0,0.0,0.0,-0.073603,...,0.0,0.0,0.0,0.210097,0.210097,0.210097,0.0,0.0,0.0,7.451556
4,2022-08-01 04:00:00,7.525156,7.329519,7.526278,7.525156,7.525156,0.0,0.0,0.0,-0.098610,...,0.0,0.0,0.0,0.210186,0.210186,0.210186,0.0,0.0,0.0,7.426546


In [ ]:
# test_preds[['yhat' , 'yhat' , 'yhat_upper']]

In [ ]:
# fig = model.plot_components(test_preds)

### Evaluation

In [24]:
def eval_metrics(y_true , y_pred):
    from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error , mean_absolute_percentage_error
    
    # MAPE hesaplama
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # r2 hesaplama
    r2 = r2_score(y_true , y_pred)
    
    # mae hesaplama
    mae = mean_absolute_error(y_true , y_pred)

    # rmse hesaplama
    mse = mean_squared_error(y_true,y_pred)**0.5
    
    print(f"""
          Mape Score : {mape}
          R2 Score : {r2}
          MAE Score : {mae}
          MSE Score : {mse}
          """)
    
def eval_plot(y_true , y_pred):
    tests = pd.DataFrame(data = y_true , columns=['Real Values'] , index = X_test[:-24].index)
    preds = pd.DataFrame(data = y_pred , columns=['Predicts'] , index = future_data[:-24].index)
    compare = pd.concat([tests[:-24], preds] , axis= 1)
    print(compare.plot())
    
def eval_df (y_true , y_pred):
    compare = pd.DataFrame({'Real Values': y_true, 'Predicts': y_pred}, index=future_data[:-24].index)
    print(compare)
    
def create_submission(future_preds, num):
    submission_df = pd.DataFrame({'Tarih': future_data.index, 'Dağıtılan Enerji (MWh)': future_preds})
    filename = 'submission{}.csv'.format(num)
    submission_df.to_csv(filename, index=False)
    globals()['submission{}'.format(num)] = submission_df
    
def preds_plot(data , future_data , target_data):
    # Gerçek değerleri mavi renkte çizdir
    plt.plot(data, color='blue', label='Gerçek Değerler')

    # Tahmin edilen değerleri yeşil renkte çizdir
    plt.plot(future_data, color='green', label='Tahminler')
    
    # Hedef ayın değerleri kırmızı renkte çizdir
    plt.plot(target_data, color='red', label='Gerçek Değerler')

    # Eksenleri ve grafik başlığını belirle
    plt.title('Gerçek Değerler ve Tahminler')
    plt.xlabel('Saat')
    plt.ylabel('Değer')
    plt.legend()

    # Grafikleri göster
    plt.show()

In [ ]:
# np.expm1(test_preds['yhat_upper'])

In [ ]:
# np.expm1(test_preds['yhat'])

In [ ]:
# test_preds['yhat_upper'] = np.expm1(test_preds['yhat_upper'])
# future_data['Dağıtılan Enerji (MWh)'] = test_preds['yhat_upper'].values

In [ ]:
# train['Dağıtılan Enerji (MWh)'] = np.expm1(train['Dağıtılan Enerji (MWh)'])

In [25]:
target_data = train[train.Tarih.dt.month == 8]

In [26]:
target_data['Dağıtılan Enerji (MWh)'] = np.expm1(target_data['Dağıtılan Enerji (MWh)'])

In [27]:
target_data = target_data[['Tarih' , 'Dağıtılan Enerji (MWh)']].set_index('Tarih')
target_data

,Dağıtılan Enerji (MWh)
Tarih,
2018-08-01 00:00:00,2106.796778
2018-08-01 01:00:00,1936.871633
2018-08-01 02:00:00,1808.523094
2018-08-01 03:00:00,1725.237068
2018-08-01 04:00:00,1680.685169
...,...
2021-08-31 19:00:00,2538.809761
2021-08-31 20:00:00,2572.939106
2021-08-31 21:00:00,2484.682803


In [ ]:
# future_data['Dağıtılan Enerji (MWh)'] = test_preds['yhat_upper']
# future_data

In [31]:
test_preds['yhat']

0      7.602585
1      7.525975
2      7.476767
3      7.451556
4      7.426546
         ...   
739    7.790251
740    7.779416
741    7.767438
742    7.740436
743    7.683152
Name: yhat, Length: 744, dtype: float64

In [32]:
# future_data['Dağıtılan Enerji (MWh)'] = np.expm1(future_data['Dağıtılan Enerji (MWh)'])
# future_data.Tarih = future_data.Tarih.astype('datetime64')
# future_data.set_index('Tarih', inplace=True , drop=True)
future_data['Dağıtılan Enerji (MWh)'] = np.expm1(test_preds['yhat'])
future_data

,Dağıtılan Enerji (MWh)
Tarih,
2022-08-01 00:00:00,NaN
2022-08-01 01:00:00,NaN
2022-08-01 02:00:00,NaN
2022-08-01 03:00:00,NaN
2022-08-01 04:00:00,NaN
...,...
2022-08-31 19:00:00,NaN
2022-08-31 20:00:00,NaN
2022-08-31 21:00:00,NaN


In [33]:
future_data['Dağıtılan Enerji (MWh)'] = np.expm1(test_preds['yhat'])
future_data

,Dağıtılan Enerji (MWh)
Tarih,
2022-08-01 00:00:00,NaN
2022-08-01 01:00:00,NaN
2022-08-01 02:00:00,NaN
2022-08-01 03:00:00,NaN
2022-08-01 04:00:00,NaN
...,...
2022-08-31 19:00:00,NaN
2022-08-31 20:00:00,NaN
2022-08-31 21:00:00,NaN


In [ ]:
# future_data = future_data.set_index('Tarih')
future_data['Dağıtılan Enerji (MWh)'] = test_preds['yhat']
future_data

In [ ]:
# train = train[['Tarih', 'Dağıtılan Enerji (MWh)']].set_index('Tarih', drop=True)
train['Dağıtılan Enerji (MWh)'] = np.expm1(train['Dağıtılan Enerji (MWh)'])
train

In [ ]:
# train = train[['Tarih' , 'Dağıtılan Enerji (MWh)']].set_index('Tarih')
# train['Dağıtılan Enerji (MWh)'] = np.expm1(train['Dağıtılan Enerji (MWh)'])
train = train[['Tarih' , 'Dağıtılan Enerji (MWh)']].set_index('Tarih',drop=True)
train

In [ ]:
target_data.mean()

In [ ]:
future_data.mean()

In [ ]:
train

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 8))
preds_plot(train , future_data , target_data )

In [ ]:
future_data

### Submission

In [34]:
test_preds['yhat'] = np.expm1(test_preds['yhat'])
sub['Dağıtılan Enerji (MWh)'] = test_preds['yhat'].values
sub.to_csv('PROPHET_yabancı_yhat4.csv', index=False)
sub[:20]

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,2002.367516
1,2022-08-01 01:00:00,1854.620803
2,2022-08-01 02:00:00,1765.520064
3,2022-08-01 03:00:00,1721.541776
4,2022-08-01 04:00:00,1678.994654
5,2022-08-01 05:00:00,1633.373689
6,2022-08-01 06:00:00,1643.954195
7,2022-08-01 07:00:00,1779.259117
8,2022-08-01 08:00:00,2048.143761
9,2022-08-01 09:00:00,2359.114600


In [35]:
sub['Dağıtılan Enerji (MWh)'].mean()

2235.92020094416

In [ ]:
# sub['Dağıtılan Enerji (MWh)'] += 100

In [ ]:
sub.head()

In [ ]:
sub.to_csv('PROPHET_yabancı_100.csv', index=False)
sub[:20]